In [146]:
import os
import sys
import numpy as np
import random
import time
import torch
# set gpu device
device = torch.device( "cuda" if torch.cuda.is_available() else "cpu" )
torch.use_deterministic_algorithms(True)
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F

# Getting the batch x

In [189]:
#loading in data ------------------------------------------------------------
import My_Anom_extargs as args
sys.path.insert(1, '/remote/gpu05/rueschkamp/projects/torch_datasets/')
from semi_dataset import SemiV_Dataset
from torch.utils.data import DataLoader


#starting training loader --------------------------------------


training_set = SemiV_Dataset(
                                    data_path = args.data_path,
                                    signal_origin= "qcd",
                                    usage= "training",
                                    number_constit= args.n_constit,
                                    number_of_jets= 1e2,
                                    ratio=0.2
                                    )

dl_training = DataLoader(training_set,batch_size=1, shuffle=True)

for i,(data,labels) in enumerate(dl_training):

    x = data

    break

In [157]:
#x = torch.randint(0,20,[128, 3, 50])
x = torch.ones([128, 3, 50])*20

pTs= torch.ones([128, 50])*10
etas = torch.ones([128, 50])*100
phis = torch.ones([128, 50])*1000
x = torch.cat((pTs.unsqueeze(1),etas.unsqueeze(1),phis.unsqueeze(1)),axis = 1)

x = x = torch.ones([1, 3, 50])

In [158]:
x = x.to(device)

# Anomaly Augmentations

### Basic fkt

In [5]:
def rescale_pts(batch):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of pT-rescaled jets, each constituent pT is rescaled by 600, same shape as input
    '''
    batch_rscl = batch.clone()
    batch_rscl[:,0,:] = torch.nan_to_num(batch_rscl[:,0,:]/600, nan=0.0, posinf=0.0, neginf=0.0)
    return batch_rscl

def recentre_jet(batch):
    batchc = batch.clone()
    nj = batch.shape[0]
    pts = batch[:,0,:]
    etas = batch[:,1,:]
    phis = batch[:,2,:]
    nc = len( pts )
    eta_shift = torch.sum(  pts*etas  ) / torch.sum( pts )
    phi_shift = torch.sum(  pts*phis ) / torch.sum( pts )
    batchc[:,1,:] = batch[:,1,:] - eta_shift
    batchc[:,2,:] = batch[:,2,:] - phi_shift
    return batchc



#### Checking recentre

In [6]:
def recentre_jet_np(batch):
    batchc = batch.copy()
    nj = batch.shape[0]
    for i in range( nj ):
        pts = batch[i,0,:]
        etas = batch[i,1,:]
        phis = batch[i,2,:]
        nc = len( pts )
        eta_shift = np.sum( [ pts[j]*etas[j] for j in range( nc ) ] ) / np.sum( pts )
        phi_shift = np.sum( [ pts[j]*phis[j] for j in range( nc ) ] ) / np.sum( pts )
        batchc[i,1,:] = batch[i,1,:] - eta_shift
        batchc[i,2,:] = batch[i,2,:] - phi_shift
    return batchc

t = recentre_jet(x)
n = recentre_jet_np(x.cpu().numpy())

print(t.cpu().numpy() - n)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


#### Checking rescale

In [12]:


def rescale_pt_np(dataset):
    for i in range(0, dataset.shape[0]):
        dataset[i,0,:] = dataset[i,0,:]/600
    return dataset

t = rescale_pts(x).cpu()
n = rescale_pt_np(x.cpu().numpy())

print(t.numpy() - n)


[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]


## Drop Constituents

In [28]:
torch.manual_seed(42)

In [26]:
def drop_constits_jet( batch, prob=0.5 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    Dim 1 ordering: (pT, eta, phi)
    Output: batch of jets where each jet has some fraction of missing constituents
    Note: rescale pts so that the augmented jet pt matches the original
    '''
    batch_dropped = batch.clone()
    #n_nonzero = torch.sum(batch_dropped[:,0,:]>0, dim=1)
    nj = batch_dropped.shape[0]
    nc = batch_dropped.shape[2]
    torch.manual_seed(42)
    mask = torch.rand((nj, nc)) > prob
    print("first mask:",mask)
    mask = mask.int().to(device)
    print(mask)
    num_zeros_tensor = (mask == 0).sum().item()
    #print(num_zeros_tensor)
    batch_dropped = batch_dropped * mask.unsqueeze(1)

    #print(batch_dropped)
    pts = torch.sum( batch[:,0,:], axis=1 )
    pts_aug = torch.sum( batch_dropped[:,0,:], axis=1 )

    pts_aug[pts_aug == 0] = 1
    pt_rescale = pts/pts_aug

    pTs= batch_dropped[:,0,:]
    etas = batch_dropped[:,1,:]
    phis = batch_dropped[:,2,:]
    pTs *= pt_rescale.unsqueeze(1)
    
    batch_dropped = torch.cat((pTs.unsqueeze(1),etas.unsqueeze(1),phis.unsqueeze(1)),axis = 1)

    #print(batch_dropped.size())
    return recentre_jet( batch_dropped )


In [36]:
def drop_constits_jet_np( batch, prob=0.5 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    Dim 1 ordering: (pT, eta, phi)
    Output: batch of jets where each jet has some fraction of missing constituents
    Note: rescale pts so that the augmented jet pt matches the original
    '''
    batchc = batch.copy()
    nj = batchc.shape[0]
    nc = batchc.shape[2]
    nzs = np.array( [ np.where( batchc[:,0,:]>0.0 )[0].shape[0] for i in range(len(batch)) ] )

    torch.manual_seed(42)
    mask = torch.rand((nj, nc)) > prob
    print("first mask:",mask)
    mask = mask.int()
    mask = mask.numpy()
    print(mask)
    num_zeros_array = ((np.array(mask)) == 0).sum()
    #print(num_zeros_array)

    for i in range( nj ):
        for j in range( nc ):
            if mask[i][j]==0:
                batchc[i,:,j] = np.array([0.0,0.0,0.0])
    pts = np.sum( batch[:,0,:], axis=1 )
    pts_aug = np.sum( batchc[:,0,:], axis=1 )
    pt_rescale = [ pts[i]/pts_aug[i] for i in range(nj) ]
    for i in range(nj):
        batchc[i,0,:] = batchc[i,0,:]*pt_rescale[i]
    return recentre_jet_np( batchc )

x=x.to(device)
t = drop_constits_jet(x,0.9)
n = drop_constits_jet_np(x.cpu().numpy(),0.9)


print(x.size())
print(t)
print(n)

first mask: tensor([[False,  True, False,  True, False, False, False, False,  True, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False,  True, False, False,
         False, False, False,  True, False, False, False, False, False, False]])
tensor([[0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
         0, 0]], device='cuda:0', dtype=torch.int32)
first mask: tensor([[False,  True, False,  True, False, False, False, False,  True, False,
          True, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False, False, False, False, False, False,  True, False, False,
         False, False, False,  True, False, Fals

In [28]:
def drop_constits_jet_ordered( batch, prob=0.5 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    Dim 1 ordering: (pT, eta, phi)
    Output: batch of jets where each jet has some fraction of missing constituents
    Note: rescale pts so that the augmented jet pt matches the original
    '''
    batch_dropped = batch.clone()
    #n_nonzero = torch.sum(batch_dropped[:,0,:]>0, dim=1)
    nj = batch_dropped.shape[0]
    nc = batch_dropped.shape[2]
    torch.manual_seed(42)
    mask = torch.rand((nj, nc)) > prob
    mask = mask.int().to(device)
    num_zeros_tensor = (mask == 0).sum().item()
    #print(num_zeros_tensor)
    batch_dropped = batch_dropped * mask.unsqueeze(1)

    #print(batch_dropped)
    pts = torch.sum( batch[:,0,:], axis=1 )
    pts_aug = torch.sum( batch_dropped[:,0,:], axis=1 )

    pts_aug[pts_aug == 0] = 1
    pt_rescale = pts/pts_aug

    pTs= batch_dropped[:,0,:]
    pTs *= pt_rescale.unsqueeze(1)
    pTs, indices = torch.sort(pTs, dim=1, descending=True) # Ordering pTs
    etas = batch_dropped[:,1,:]
    etas = etas.gather(dim=1,index=indices)
    phis = batch_dropped[:,2,:]
    phis = phis.gather(dim=1,index=indices)

    
    batch_dropped = torch.cat((pTs.unsqueeze(1),etas.unsqueeze(1),phis.unsqueeze(1)),axis = 1)

    #print(batch_dropped.size())
    return recentre_jet( batch_dropped )

In [29]:
y = drop_constits_jet_ordered(x)
z= drop_constits_jet(x)
print(y)
print(z)

first mask: tensor([[ True,  True, False,  True, False,  True, False,  True,  True, False,
          True,  True,  True,  True,  True, False,  True,  True, False,  True,
         False, False, False,  True, False, False, False, False,  True, False,
          True, False,  True,  True, False,  True,  True,  True,  True, False,
          True, False,  True,  True,  True, False,  True,  True,  True, False]])
tensor([[1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
         0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1,
         1, 0]], device='cuda:0', dtype=torch.int32)
tensor([[[ 1.1189e+02,  3.7674e+01,  1.5382e+01,  3.1732e+00,  2.1747e+00,
           1.9729e+00,  1.6918e+00,  1.6153e+00,  1.3249e+00,  8.8348e-01,
           4.9134e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
 

## pT reweight


In [13]:
def pt_reweight_jet( batch, beta=1.5 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    Dim 1 ordering: (pT, eta, phi)
    Output: batch of jets where the pt of the constituents in each jet has has been re-weighted by some power
    Note: rescale pts so that the augmented jet pt matches the original
    '''
    batchc = batch.clone()

    etas = batchc[:,1,:]
    phis = batchc[:,2,:]
    batchc = batchc[:,0,:]**beta
    pts = torch.sum( batch[:,0,:], axis=1 )
    pts_aug = torch.sum( batchc, axis=1 )

    pts_aug[pts_aug == 0] = 1
    pt_rescale =  pts/pts_aug
    pTs = pt_rescale.unsqueeze(-1)* batchc
    #print(pTs)

    jet = torch.cat((pTs.unsqueeze(1),etas.unsqueeze(1),phis.unsqueeze(1)),axis = 1)
    return recentre_jet( jet.float() )

In [14]:
def pt_reweight_jet_np( batch, beta=1.5 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    Dim 1 ordering: (pT, eta, phi)
    Output: batch of jets where the pt of the constituents in each jet has has been re-weighted by some power
    Note: rescale pts so that the augmented jet pt matches the original
    '''
    batchc = batch.copy()
    nj = batchc.shape[0]
    nc = batchc.shape[2]
    for i in range( nj ):
        for j in range( nc ):
            batchc[i,0,j] = batch[i,0,j]**beta
    pts = np.sum( batch[:,0,:], axis=1 )
    pts_aug = np.sum( batchc[:,0,:], axis=1 )
    pt_rescale = [ pts[i]/pts_aug[i] for i in range(nj) ]
    for i in range(nj):
        batchc[i,0,:] = batchc[i,0,:]*pt_rescale[i]
    #print(batchc[:,0,:])
    return recentre_jet_np( batchc )

t = pt_reweight_jet(x)
n = pt_reweight_jet_np(x.cpu().numpy())

print(t.cpu().numpy() - n)
print(t - torch.Tensor(n).to(device))
#print(t - x)

[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],



In [ ]:
def pt_reweight_jet_ordered( batch, beta=1.5 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    Dim 1 ordering: (pT, eta, phi)
    Output: batch of jets where the pt of the constituents in each jet has has been re-weighted by some power
    Note: rescale pts so that the augmented jet pt matches the original
    '''
    batchc = batch.clone()

    etas = batchc[:,1,:]
    phis = batchc[:,2,:]
    batchc = batchc[:,0,:]**beta
    pts = torch.sum( batch[:,0,:], axis=1 )
    pts_aug = torch.sum( batchc, axis=1 )

    pts_aug[pts_aug == 0] = 1
    pt_rescale =  pts/pts_aug
    pTs = pt_rescale.unsqueeze(-1)* batchc
    #print(pTs)
    pTs, indices = torch.sort(pTs, dim=1, descending=True) # Ordering pTs
    etas = batch_dropped[:,1,:]
    etas = etas.gather(dim=1,index=indices)
    phis = batch_dropped[:,2,:]
    phis = phis.gather(dim=1,index=indices)

    jet = torch.cat((pTs.unsqueeze(1),etas.unsqueeze(1),phis.unsqueeze(1)),axis = 1)
    return recentre_jet( jet.float() )

## adding stuff


In [23]:
def add_jets(batch):
    batch_filled = batch.clone()
    n_constit = batch_filled.shape[2]
    n_nonzero = torch.sum(batch_filled[:,0,:]>0, dim=1) #number of zero constituents
    print(n_nonzero)
    n_split = torch.min(torch.stack([n_nonzero, n_constit-n_nonzero], dim=1), dim=1).values
    print(n_split)

In [24]:
y = add_jets(x)

tensor([33, 26, 21, 25, 22, 29, 23, 24, 27, 27, 27, 21, 24, 22, 22, 28, 24, 29,
        26, 31, 26, 27, 27, 25, 23, 26, 27, 23, 26, 23, 27, 25, 29, 25, 27, 20,
        22, 26, 33, 20, 28, 27, 25, 26, 16, 22, 23, 26, 30, 27, 28, 21, 27, 25,
        23, 26, 22, 24, 26, 25, 24, 23, 29, 21, 29, 25, 23, 20, 26, 20, 30, 21,
        28, 25, 27, 17, 26, 22, 19, 25, 18, 24, 26, 29, 25, 28, 21, 26, 17, 26,
        25, 26, 23, 20, 23, 27, 28, 21, 24, 22, 27, 24, 12, 27, 25, 24, 24, 28,
        28, 23, 26, 25, 26, 23, 32, 32, 28, 24, 29, 28, 29, 23, 24, 22, 26, 26,
        28, 24])
tensor([17, 24, 21, 25, 22, 21, 23, 24, 23, 23, 23, 21, 24, 22, 22, 22, 24, 21,
        24, 19, 24, 23, 23, 25, 23, 24, 23, 23, 24, 23, 23, 25, 21, 25, 23, 20,
        22, 24, 17, 20, 22, 23, 25, 24, 16, 22, 23, 24, 20, 23, 22, 21, 23, 25,
        23, 24, 22, 24, 24, 25, 24, 23, 21, 21, 21, 25, 23, 20, 24, 20, 20, 21,
        22, 25, 23, 17, 24, 22, 19, 25, 18, 24, 24, 21, 25, 22, 21, 24, 17, 24,
        25, 24, 23, 20,

# Physical Augmentations

## Collinear fill jets 

In [176]:
def collinear_fill_jets_fast(batch):
    '''
    Fill as many of the zero-padded entries with collinear splittings
    of the constituents by splitting each constituent at most once.
    Parameters
    ----------
    batch : torch.Tensor
        batch of jets with zero-padding
    Returns
    -------
    batch_filled : torch.Tensor
        batch of jets with collinear splittings
    '''
    batch_filled = batch.clone()
    n_constit = batch_filled.shape[2]
    n_nonzero = torch.sum(batch_filled[:,0,:]>0, dim=1)
    
    n_split = torch.min(torch.stack([n_nonzero, n_constit-n_nonzero], dim=1), dim=1).values
    idx_flip = torch.where(n_nonzero != n_split)[0]
    mask_split = (batch_filled[:,0,:] != 0)
    
    mask_split[idx_flip] = torch.flip(mask_split[idx_flip].float(), dims=[1]).bool()

    #print(mask_split)
    mask_split[idx_flip] = ~mask_split[idx_flip]
    r_split = torch.rand(size=mask_split.shape, device=batch.device)
    
    a = r_split*mask_split*batch_filled[:,0,:]
    b = (1-r_split)*mask_split*batch_filled[:,0,:]
    c = ~mask_split*batch_filled[:,0,:]
    batch_filled[:,0,:] = a+c+torch.flip(b, dims=[1])
    batch_filled[:,1,:] += torch.flip(mask_split*batch_filled[:,1,:], dims=[1])
    batch_filled[:,2,:] += torch.flip(mask_split*batch_filled[:,2,:], dims=[1])
    return batch_filled

In [177]:
def collinear_fill_jets_np( batch ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of jets with collinear splittings, the function attempts to fill as many of the zero-padded args.nconstit
    entries with collinear splittings of the constituents by splitting each constituent at most once, same shape as input
    '''
    batchb = batch.copy()
    nc = batch.shape[2]
    nzs = np.array( [ np.where( batch[:,0,:][i]>0.0)[0].shape[0] for i in range(len(batch)) ] )
    for k in range(len(batch)):
        nzs1 = np.max( [ nzs[k], int(nc/2) ] )
        zs1 = int(nc-nzs1)
        els = np.random.choice( np.linspace(0,nzs1-1,nzs1), size=zs1, replace=False )
        rs = np.random.uniform( size=zs1 )
        for j in range(zs1):
            batchb[k,0,int(els[j])] = rs[j]*batch[k,0,int(els[j])]
            batchb[k,0,int(nzs[k]+j)] = (1-rs[j])*batch[k,0,int(els[j])]
            batchb[k,1,int(nzs[k]+j)] = batch[k,1,int(els[j])]
            batchb[k,2,int(nzs[k]+j)] = batch[k,2,int(els[j])]
    return batchb

In [178]:
t = collinear_fill_jets_fast(x)
n = collinear_fill_jets_np(x.cpu().numpy())

print(t.cpu().numpy() - n)

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]]]


## Rotations 

### Only phi

In [ ]:
def rotate_jets(batch):

    rot_batch = batch # is this right when it should stay on gpu?
    batch_size = batch.size(0)
    constit = batch.size(2)

    rotate_tensor = torch.rand([batch_size,constit]) * 2 * np.pi #creating the array of random rotations

    rot_batch[:,2,:] =+ np.pi # shifting the phi tensor to make use of the % function
    rot_batch[:,2,:] += rotate_tensor
    rot_batch[:,2,:] %= 2 * np.pi # getting it in the same output range
    rot_batch[:,2,:] =- np.pi # shifting back

    return rot_batch

### Bary does it all

In [165]:
import torch


def rotate_jets(batch, ra ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of jets rotated independently in eta-phi, same shape as input
    '''
    device = batch.device
    batch_size = batch.size(0)

    torch.manual_seed(42)
    rot_angle = ra
    #rot_angle = torch.rand(batch_size, device="cpu") * 2 * np.pi
    c = torch.cos(rot_angle)
    s = torch.sin(rot_angle)
    o = torch.ones_like(rot_angle)
    z = torch.zeros_like(rot_angle)

    #print(o.shape)

    rot_matrix = torch.stack([
    torch.stack([o, z, z], dim=0),
    torch.stack([z, c, s], dim=0),
    torch.stack([z, -s, c], dim=0)], dim=1) # (3, 3, batch_size]

    #print(rot_matrix[:,:,0])

    return torch.einsum('ijk,lji->ilk', batch, rot_matrix)


In [166]:
def rotate_jets_np( batch , ra ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of jets rotated independently in eta-phi, same shape as input
    '''

    rot_angle = ra

    torch.manual_seed(42)
    #rot_angle = (torch.rand(128, device="cpu") * 2 * np.pi).cpu().numpy()
    c = np.cos(rot_angle)
    s = np.sin(rot_angle)
    o = np.ones_like(rot_angle)
    z = np.zeros_like(rot_angle)
    rot_matrix = np.array([[o, z, z], [z, c, -s], [z, s, c]]) # (3, 3, batchsize)
    return np.einsum('ijk,lji->ilk', batch, rot_matrix)

In [167]:
rot_angle = torch.rand(batch_size, device="cpu") * 2 * np.pi

t = rotate_jets(x.cpu(),rot_angle).cpu().numpy()
n = rotate_jets_np(x.cpu().numpy(),rot_angle.numpy())

print((t- n))

[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
   0. 0. 0. 0.]]]


In [163]:
batch_size = 2
rot_angle = torch.ones(batch_size, device="cpu")

c = torch.Tensor([1])#torch.cos(rot_angle)
s = torch.Tensor([2])#torch.sin(rot_angle)
o = torch.Tensor([3])#torch.ones_like(rot_angle)
z = torch.Tensor([4])#torch.zeros_like(rot_angle)

rot_matrix = torch.stack([
torch.stack([o, z, z], dim=0),
torch.stack([z, c, s], dim=0),
torch.stack([z, -s, c], dim=0)], dim=1) # (3, 3, batch_size]

rot_matrix_np = np.array([[o.numpy(), z.numpy(), z.numpy()], [z.numpy(), c.numpy(), -s.numpy()], [z.numpy(), s.numpy(), c.numpy()]]) # (3, 3, batchsize)

In [164]:
print(rot_matrix.numpy())
print(rot_matrix_np)

[[[ 3.]
  [ 4.]
  [ 4.]]

 [[ 4.]
  [ 1.]
  [-2.]]

 [[ 4.]
  [ 2.]
  [ 1.]]]
[[[ 3.]
  [ 4.]
  [ 4.]]

 [[ 4.]
  [ 1.]
  [-2.]]

 [[ 4.]
  [ 2.]
  [ 1.]]]


## Distort Jets 

In [169]:
import torch
torch.manual_seed(42)

def distort_jets(batch, strength=0.1, pT_clip_min=0.1):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of jets with each constituents position shifted independently, shifts drawn from normal with mean 0, std strength/pT, same shape as input
    '''
    #strength = torch.Tensor(strength).to(device)
    pT = batch[:, 0]  # (batchsize, n_constit)
    shift_eta = torch.nan_to_num(
        #strength * torch.randn(batch.shape[0], batch.shape[2]).to(device) / pT.clip(min=pT_clip_min).to(device),
        strength * torch.ones(batch.shape[0], batch.shape[2]).to(device) / pT.clip(min=pT_clip_min).to(device),
        posinf=0.0,
        neginf=0.0,
    ).to(device)  # * mask
    shift_phi = torch.nan_to_num(
        #strength * torch.randn(batch.shape[0], batch.shape[2]).to(device) / pT.clip(min=pT_clip_min).to(device),
        strength * torch.ones(batch.shape[0], batch.shape[2]).to(device) / pT.clip(min=pT_clip_min).to(device),
        posinf=0.0,
        neginf=0.0,
    ).to(device)  # * mask
    shift = torch.stack([torch.zeros((batch.shape[0], batch.shape[2])).to(device), shift_eta, shift_phi], 1)
    return batch + shift


In [174]:
def distort_jets_np( batch, strength=0.1, pT_clip_min=0.1 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of jets with each constituents position shifted independently, shifts drawn from normal with mean 0, std strength/pT, same shape as input
    '''
    pT = batch[:,0]   # (batchsize, n_constit)
    #shift_eta = np.nan_to_num( strength * np.random.randn(batch.shape[0], batch.shape[2]) / pT.clip(min=pT_clip_min), posinf = 0.0, neginf = 0.0 )# * mask
    #shift_phi = np.nan_to_num( strength * np.random.randn(batch.shape[0], batch.shape[2]) / pT.clip(min=pT_clip_min), posinf = 0.0, neginf = 0.0 )# * mask

    shift_eta = np.nan_to_num(strength * np.ones((batch.shape[0], batch.shape[2]), dtype=np.float64) / pT.clip(min=pT_clip_min), posinf=0.0, neginf=0.0)
    shift_phi = np.nan_to_num(strength * np.ones((batch.shape[0], batch.shape[2]), dtype=np.float64) / pT.clip(min=pT_clip_min), posinf=0.0, neginf=0.0)

    shift = np.stack( [ np.zeros( (batch.shape[0], batch.shape[2]) ), shift_eta, shift_phi ], 1)
    return batch + shift

In [175]:
t = distort_jets(x).cpu()
n = distort_jets_np(x.cpu().numpy())

print(t.numpy() - n)

[[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
   0.00000000e+00 0.00000000e+00]
  [2.38418578e-08 2.38418578e-08 2.38418578e-08 2.38418578e-08
   2.38418578e-08 2.38418578e-08 2.38418578e-08 2.38418578e-08
   2.38418578e-08 2.38418578e-08 2.38418578e-08 2.38418578e-08
   2.38418578e-08 2.3

## Translate Jets 

In [196]:
import torch

def ptp(input, dim=None, keepdim=False):
    if dim is None:
        return input.max() - input.min()
    return input.max(dim, keepdim).values - input.min(dim, keepdim).values


def translate_jets(batch, width=1.0):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of eta-phi translated jets, same shape as input
    '''

    device = batch.device
    mask = (batch[:, 0] > 0)  # 1 for constituents with non-zero pT, 0 otherwise
    ptp_eta = ptp(batch[:, 1, :], dim=-1, keepdim=True)  # ptp = 'peak to peak' = max - min DOUBLE CHECKED
    ptp_phi = ptp(batch[:, 2, :], dim=-1, keepdim=True)  # ptp = 'peak to peak' = max - min
    low_eta = -width * ptp_eta
    high_eta = +width * ptp_eta
    low_phi = torch.max(-width * ptp_phi, -torch.tensor(np.pi, device=device) - torch.amin(batch[:, 2, :], dim=-1, keepdim=True))
    high_phi = torch.min(+width * ptp_phi, +torch.tensor(np.pi, device=device) - torch.amax(batch[:, 2, :], dim=-1, keepdim=True)) #DOUBLE CHECKED
    shift_eta = mask * torch.rand((batch.shape[0], 1), device=device) * (high_eta - low_eta) + low_eta
    shift_phi = mask * torch.rand((batch.shape[0], 1), device=device) * (high_phi - low_phi) + low_phi
    shift = torch.stack([torch.zeros((batch.shape[0], batch.shape[2]), device=device), shift_eta, shift_phi], dim=1)
    print(shift)
    shifted_batch = batch + shift
    return shifted_batch



In [200]:
def translate_jets_np( batch, width=1.0 ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of eta-phi translated jets, same shape as input
    '''
    
    mask = (batch[:,0] > 0) # 1 for constituents with non-zero pT, 0 otherwise
    ptp_eta  = np.ptp(batch[:,1,:], axis=-1, keepdims=True) # ptp = 'peak to peak' = max - min
    ptp_phi  = np.ptp(batch[:,2,:], axis=-1, keepdims=True) # ptp = 'peak to peak' = max - min
    low_eta  = -width*ptp_eta
    high_eta = +width*ptp_eta
    low_phi  = np.maximum(-width*ptp_phi, -np.pi-np.amin(batch[:,2,:], axis=1).reshape(ptp_phi.shape))
    high_phi = np.minimum(+width*ptp_phi, +np.pi-np.amax(batch[:,2,:], axis=1).reshape(ptp_phi.shape))
    print(high_phi)
    #shift_eta = (mask * torch.rand((batch.shape[0], 1), device=device) * (high_eta - low_eta)).cpu().numpy()
    #shift_phi = (mask * torch.rand((batch.shape[0], 1), device=device) * (high_phi - low_phi)).cpu().numpy()

    shift_eta = mask*np.random.uniform(low=low_eta, high=high_eta, size=(batch.shape[0], 1))
    shift_phi = mask*np.random.uniform(low=low_phi, high=high_phi, size=(batch.shape[0], 1))
    shift = np.stack([np.zeros((batch.shape[0], batch.shape[2])), shift_eta, shift_phi], 1)
    print(shift)
    shifted_batch = batch+shift
    return shifted_batch


As the random function can not be compared i show here that they are doing the same using the mean and the variance.

In [207]:
batch = x
ptp_eta = ptp(batch[:, 1, :], dim=-1, keepdim=True) 
batch = batch.cpu()
ptp_eta_np  = np.ptp(batch[:,1,:], axis=-1, keepdims=True) # ptp = 'peak to peak' = max - min

print(ptp_eta)
print(ptp_eta_np)

import numpy as np
import torch

# Set the seed for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Parameters for uniform distribution
low = 0.0
high = 1.0
size = (200, 3000)  # Shape of the output array

# Generate random numbers using np.random.uniform()
np_uniform = np.random.uniform(low=low, high=high, size=size)
print("NumPy random numbers:\n", np_uniform)
print(np.mean(np_uniform),np.var(np_uniform))
# Generate random numbers using torch.rand()
torch_uniform = low + (high - low) * torch.rand(size)
print("PyTorch random numbers:\n", torch_uniform)
print(torch.mean(torch_uniform),torch.var(torch_uniform))

tensor([[1.1898]], device='cuda:0')
tensor([[1.1898]])
NumPy random numbers:
 [[0.5488135  0.71518937 0.60276338 ... 0.32361371 0.81354502 0.69740038]
 [0.4139625  0.6296183  0.77858426 ... 0.69652287 0.48369661 0.33955073]
 [0.37479353 0.42868598 0.68305662 ... 0.06915139 0.75806398 0.932006  ]
 ...
 [0.50755403 0.45498658 0.00269646 ... 0.30199667 0.45259287 0.35421085]
 [0.30057139 0.19176931 0.2468228  ... 0.45199921 0.74777253 0.16047437]
 [0.86403991 0.54683827 0.34085736 ... 0.75526211 0.21382493 0.72614352]]
0.4999276664433702 0.08339419508947447
PyTorch random numbers:
 tensor([[0.4963, 0.7682, 0.0885,  ..., 0.2716, 0.3017, 0.0812],
        [0.3791, 0.4516, 0.2695,  ..., 0.0580, 0.5345, 0.3919],
        [0.9744, 0.2747, 0.1823,  ..., 0.0410, 0.7614, 0.3119],
        ...,
        [0.1531, 0.0114, 0.2678,  ..., 0.0205, 0.8479, 0.0535],
        [0.5164, 0.8979, 0.1201,  ..., 0.9679, 0.2169, 0.3057],
        [0.9008, 0.1807, 0.2671,  ..., 0.9723, 0.3218, 0.6172]])
tensor(0.5004) t

## normalize

In [86]:
def normalise_pts(batch):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of pT-normalised jets, pT in each jet sums to 1, same shape as input
    '''
    batch_norm = batch.clone()
    batch_norm[:, 0, :] = torch.nan_to_num(batch_norm[:, 0, :] / torch.sum(batch_norm[:, 0, :], dim=1)[:, None], posinf=0.0, neginf=0.0)
    return batch_norm

In [87]:
def normalise_pts_np( batch ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of pT-normalised jets, pT in each jet sums to 1, same shape as input
    '''
    batch_norm = batch.copy()
    batch_norm[:,0,:] = np.nan_to_num(batch_norm[:,0,:]/np.sum(batch_norm[:,0,:], axis=1)[:, np.newaxis], posinf = 0.0, neginf = 0.0 )
    return batch_norm

In [89]:
t = normalise_pts(x).cpu()
n = normalise_pts_np(x.cpu().numpy())

print(t.numpy() - n)

[[[-1.4901161e-08 -1.4901161e-08 -1.4901161e-08 -7.4505806e-09
   -7.4505806e-09 -7.4505806e-09 -3.7252903e-09  0.0000000e+00
   -1.8626451e-09 -3.7252903e-09 -1.8626451e-09 -1.8626451e-09
   -1.8626451e-09 -1.8626451e-09 -1.8626451e-09 -9.3132257e-10
   -9.3132257e-10 -9.3132257e-10 -9.3132257e-10 -9.3132257e-10
   -9.3132257e-10 -4.6566129e-10 -4.6566129e-10 -4.6566129e-10
   -4.6566129e-10 -4.6566129e-10  0.0000000e+00 -4.6566129e-10
   -4.6566129e-10 -4.6566129e-10 -4.6566129e-10 -2.3283064e-10
   -4.6566129e-10 -2.3283064e-10  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.

## rescale

In [90]:
def rescale_pts(batch):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of pT-rescaled jets, each constituent pT is rescaled by 600, same shape as input
    '''
    batch_rscl = batch.clone()
    batch_rscl[:,0,:] = torch.nan_to_num(batch_rscl[:,0,:]/600, nan=0.0, posinf=0.0, neginf=0.0)
    return batch_rscl

In [91]:
def rescale_pts_np( batch ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of pT-rescaled jets, each constituent pT is rescaled by 600, same shape as input
    '''
    batch_rscl = batch.copy()
    batch_rscl[:,0,:] = np.nan_to_num(batch_rscl[:,0,:]/600, posinf = 0.0, neginf = 0.0 )
    return batch_rscl


In [96]:
t = rescale_pts(x).cpu()
n = rescale_pts_np(x.cpu().numpy())


print(t)
print(t.numpy() - n)

tensor([[[ 5.2316e-02,  4.8924e-02,  3.9041e-02,  2.1909e-02,  2.1318e-02,
           2.0094e-02,  1.0358e-02,  1.0031e-02,  8.9396e-03,  8.6376e-03,
           8.4717e-03,  6.4226e-03,  5.9001e-03,  5.8193e-03,  5.0375e-03,
           4.7770e-03,  4.4699e-03,  4.0331e-03,  3.4782e-03,  3.3408e-03,
           2.5714e-03,  2.3699e-03,  2.3166e-03,  2.1685e-03,  1.8395e-03,
           1.6398e-03,  1.5695e-03,  1.4866e-03,  1.4825e-03,  1.4456e-03,
           1.2463e-03,  1.2226e-03,  1.1302e-03,  5.5457e-04,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
           0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
         [-7.9058e-02,  6.1648e-04,  6.2118e-03,  5.7302e-03,  1.8753e-02,
          -3.3219e-04, -9.8502e-02,  1.3584e-01, -2.4150e-02, -4.8453e-02,
           3.8021e-03, -2.3874e-02,  4.9717e-01, -4.8027e-02, -6.9739e-02,
           1.0976e-01, -

## Crop it 

In [93]:
def crop_jets( batch, nc ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of cropped jets, each jet is cropped to nc constituents, shape (batchsize, 3, nc)
    '''
    batch_crop = batch.clone()
    return batch_crop[:,:,0:nc]

In [94]:
def crop_jets_np( batch, nc ):
    '''
    Input: batch of jets, shape (batchsize, 3, n_constit)
    dim 1 ordering: (pT, eta, phi)
    Output: batch of cropped jets, each jet is cropped to nc constituents, shape (batchsize, 3, nc)
    '''
    batch_crop = batch.copy()
    return batch_crop[:,:,0:nc]

In [97]:
t = crop_jets(x,3).cpu()
n = crop_jets_np(x.cpu().numpy(),3)

print(t.numpy() - n)

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
